# Exploring the Global Summary of the Year Dataset

## First Goal: See how much of the data (.csv files) contains temperature information

Would like to explore trends and answer questions such as: How many countries are seeing increased maximum or average temperatures over time? What is the average trend line of temperature increase? What is the KDE for the trend distribution? Can a dashboard be made to visualize this geographically?
            
            

### Load Libraries

In [101]:
import pandas as pd
import numpy as np
import os
import random
from datetime import datetime
import re

### Define Functions

In [3]:
def percent_missing(dataframe, groupby_col, col_name):
    groupby = dataframe.groupby(dataframe[groupby_col])[col_name].sum()
    proportion_non_na = round(len(groupby[groupby != 0])/len(groupby)*100,2)
    print(f"{proportion_non_na}% of the {groupby_col}s have {col_name} data")

### Convert all 80k+ CSVs into one master CSV (only run once)
   
   #csv_data_files = os.listdir('Global Summary of the Year NCEI Data')
len(csv_data_files)


    #os.chdir('Global Summary of the Year NCEI Data')
df = pd.concat(map(pd.read_csv, csv_data_files), ignore_index=True)

#Change directory 
os.chdir("..")
os.path.abspath(os.curdir)

df.to_csv('GSOY_Master_File.csv')


In [4]:
#Check current working directory

os.path.abspath(os.curdir)

'C:\\Users\\dminkler002\\OneDrive - Guidehouse\\Documents\\Training\\Climate Project\\ClimateProjectGSOY'

### Read in master csv

In [5]:
df_master = pd.read_csv("GSOY_Master_File.csv")

#Preview the CSV
df_master

C:\Users\dminkler002\AppData\Local\Temp\ipykernel_3936\3031860136.py:1: DtypeWarning: Columns (80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200,202,204,206,208,210,212,214,216,217) have mixed types. Specify dtype option on import or set low_memory=False.
  df_master = pd.read_csv("GSOY_Master_File.csv")


,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,...,MN09_ATTRIBUTES,MX05,MX05_ATTRIBUTES,MX06,MX06_ATTRIBUTES,MX07,MX07_ATTRIBUTES,MX08,MX08_ATTRIBUTES,The FTP site ftp://ftp.nodc.noaa.gov/ has been decommissioned
0,0,USC00401946,1949.0,35.51667,-85.28333,264.9,"COLLEGE JUNCTION, TN US",112.0,0,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USC00401946,1950.0,35.51667,-85.28333,264.9,"COLLEGE JUNCTION, TN US",127.0,0,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,USC00401946,1951.0,35.51667,-85.28333,264.9,"COLLEGE JUNCTION, TN US",105.0,0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,USC00401946,1952.0,35.51667,-85.28333,264.9,"COLLEGE JUNCTION, TN US",91.0,0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,USC00401946,1953.0,35.51667,-85.28333,264.9,"COLLEGE JUNCTION, TN US",100.0,0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525939,2525939,MXN00003093,2009.0,27.15000,-112.15000,19.8,"SAN BRUNO, MX",13.0,m,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2525940,2525940,MXN00003093,2010.0,27.15000,-112.15000,19.8,"SAN BRUNO, MX",6.0,m,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2525941,2525941,MXN00003093,2012.0,27.15000,-112.15000,19.8,"SAN BRUNO, MX",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2525942,2525942,USC00055121,1949.0,40.24660,-105.14630,1569.7,"LONGMONT 6 NW, CO US",75.0,0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Explore the csv 

In [152]:
# List of column names
col_names  = [x for x in df_master.columns]
col_names

#Define metric of interest and metric of interest attributes
moi = 'TAVG'
moia = 'TAVG_ATTRIBUTES'

#Find proportion of stations that report the moi and the moia
moi_groupby = df_master.groupby(df_master.STATION)[moi].sum()
proportion_non_na_moi = round(len(moi_groupby[moi_groupby > 0])/len(moi_groupby)*100,2)
print(f"{proportion_non_na_moi}% of the stations have {moi} data")

moia_groupby = df_master.groupby(df_master.STATION)[moia].sum()
proportion_non_na_moia = round(len(moia_groupby[moia_groupby != 0])/len(moia_groupby)*100,2)
print(f"{proportion_non_na_moia}% of the stations have {moia} data")

37.3% of the stations have TAVG data
39.08% of the stations have TAVG_ATTRIBUTES data


#### Assume for now that TAVG_ATTRIBUTES cover all relevant TAVG data

## Subset the data to only data with the relevant MOI, then filter by date

In [153]:
#Make a list of stations
station_list_with_moi = moi_groupby[moi_groupby > 0].index.tolist()
#Subset the dataframe
df_moi = df_master[df_master['STATION'].isin(station_list_with_moi)]

In [154]:
#Convert data column to datetime dtype after converting to int
df_moi['DATE'] = df_moi['DATE'].astype(int)
df_moi['DATE'] = pd.to_datetime(df_moi['DATE'], format = '%Y')

C:\Users\dminkler002\AppData\Local\Temp\ipykernel_3936\4080105068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_moi['DATE'] = df_moi['DATE'].astype(int)
C:\Users\dminkler002\AppData\Local\Temp\ipykernel_3936\4080105068.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_moi['DATE'] = pd.to_datetime(df_moi['DATE'], format = '%Y')


In [155]:
#What is the most recent date?
max(df_moi['DATE']) #2022

#Subset DF and Check to see how many stations contain a data range of 20 years 2002 to 2022
df_moi_date = df_moi[(df_moi['DATE'] >= datetime(2002,1,1)) & (df_moi['DATE'] <= datetime(2022,1,1))]
print(round(len(df_moi_date['STATION'].unique())/len(df_master['STATION'].unique())*100),2)

#Check for missing dates b/t 2002 and 2022 


23 2


In [156]:
full_date_range =  [x.year for x in pd.date_range(start = '1/1/2002', freq = 'YS', periods = 20)] # YS = year start 

# Pick a random station to test the date range approach to check for missing values
random.seed(20)
station_list = df_moi_date['STATION'].unique()
station_dates = [x.year for x in df_moi_date.loc[df_moi_date['STATION'] == random.choice(station_list),]['DATE']]

# Identify all station names which satisfy this date range

station_list_daterange = []
for i in station_list:
    station_dates = [x.year for x in df_moi_date.loc[df_moi_date['STATION'] == i,]['DATE']]
    date_check = [y for y in full_date_range if y not in station_dates]
    if len(date_check) == 0:
        station_list_daterange.append(i)

In [157]:
len(station_list_daterange)

4225

#### If we choose a date range of 2002 to 2022, we get 1441 stations which meet and have the required dates, If we choose a date range of 2002 to 2021, we get 4225 stations which meet and have the required dates
### We will go with a date range of 2002 to 2021. Subset the df by the station list where criteria has been met

In [158]:
# Code to subset by station list 
df_moi_date_range_20y = df_moi_date.loc[df_moi_date['STATION'].isin(station_list_daterange), ]

#Filter out any date values greater than 2021
df_moi_date_range_20y = df_moi_date_range_20y[df_moi_date_range_20y['DATE'] < datetime(2022,1,1)]

#Sanity check by checking group size of all stations (should be zero)
len([x for x in df_moi_date_range_20y.groupby('STATION').size() if x != 20])


0

In [159]:
df_moi_date_range_20y

,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,...,MN09_ATTRIBUTES,MX05,MX05_ATTRIBUTES,MX06,MX06_ATTRIBUTES,MX07,MX07_ATTRIBUTES,MX08,MX08_ATTRIBUTES,The FTP site ftp://ftp.nodc.noaa.gov/ has been decommissioned
170,170,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,171,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",134.0,0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,172,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",112.0,0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,173,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,174,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",97.0,0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524916,2524916,GME00121978,2017-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",188.0,E,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524917,2524917,GME00121978,2018-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",144.0,E,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524918,2524918,GME00121978,2019-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",172.0,E,110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524919,2524919,GME00121978,2020-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",166.0,E,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Country Codes and Match with Country Names

In [160]:
# Extract the country code from the GHCN station ID using REGEX, create new columns. Extract 2 and 3 character codes 
df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'] = df_moi_date_range_20y['STATION'].str.extract(r'(\D{2})')
df_moi_date_range_20y['COUNTRY_CODE_3_CHAR'] = df_moi_date_range_20y['STATION'].str.extract(r'(\D+)')

In [161]:
# Explore the country codes found (extracted from GHCN Station ID)
df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'].unique().tolist()

print('There are {} country codes in our data set. These include {}'.format(
    len(df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'].unique()),
    df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'].unique().tolist()
        )
     )

There are 32 country codes in our data set. These include ['US', 'GM', 'SW', 'NO', 'FI', 'AS', 'SP', 'AU', 'UK', 'FR', 'FM', 'NL', 'SZ', 'CH', 'VQ', 'EN', 'GQ', 'SI', 'RI', 'IS', 'CA', 'BU', 'RO', 'HR', 'AG', 'DA', 'RM', 'RQ', 'LO', 'SF', 'BK', 'JA']


In [162]:
# Read text file with country code legend from https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily
with open("ghcn_country_codes.txt") as f:
    contents = f.readlines()

# Remove \n in all of the strings 
contents = [x.replace(' \n', '').replace('\n', '') for x in contents]

# Convert list to a dictionary 
country_names = {x[0:2]:x[3:] for x in contents}

# Convert Dictionary to pandas dataframe to make joining easier
country_names = pd.DataFrame.from_dict(country_names, orient = 'index', columns = ['country_name'])
country_names.head()




,country_name
AC,Antigua and Barbuda
AE,United Arab Emirates
AF,Afghanistan
AG,Algeria
AJ,Azerbaijan


In [164]:
set(country_names.columns) - set(df_moi_date_range_20y.columns)

{'country_name'}

In [165]:
# Join Country Names to DataFrame
df_moi_date_range_20y = df_moi_date_range_20y.merge(country_names, how='left', left_on = 'COUNTRY_CODE_2_CHAR', right_on = country_names.index)

df_moi_date_range_20y.head(5)

,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,...,MX06,MX06_ATTRIBUTES,MX07,MX07_ATTRIBUTES,MX08,MX08_ATTRIBUTES,The FTP site ftp://ftp.nodc.noaa.gov/ has been decommissioned,COUNTRY_CODE_2_CHAR,COUNTRY_CODE_3_CHAR,country_name
0,170,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,USW,United States
1,171,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",134.0,0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,USW,United States
2,172,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",112.0,0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,USW,United States
3,173,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,USW,United States
4,174,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",97.0,0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,USW,United States


## Join Reverse Geocoding data to DF

In [167]:
#Load in lat-lon df from geocoding, drop unnamed column
lat_lon_df = pd.read_csv('lat_lon_df.csv').drop('Unnamed: 0', axis = 1)

## Need to convert a dictionary which is saved as a string to an actual dictionary
## First, convert nan to 'nan' so that eval func. can be used

#Test case

#regex = re.compile('\s(nan)(?=\W)')
#print(regex.search(lat_lon_df['REVERSE_GEOCODE'][1]))

# Final solution 
lat_lon_df['REVERSE_GEOCODE'] = lat_lon_df['REVERSE_GEOCODE'].str.replace(r'\s(nan)(?=\W)', "'nan'", regex = True)

## Then convert strings to dicts
lat_lon_df['REVERSE_GEOCODE'] = lat_lon_df['REVERSE_GEOCODE'].apply(eval)

# Unpack the REVERSE_GEOCODE column to multiple columns, then join to main 20y_sub df
lat_lon_df = pd.concat([lat_lon_df.loc[:,'lat_lon'], lat_lon_df['REVERSE_GEOCODE'].apply(pd.Series)], axis=1)

lat_lon_df

,lat_lon,city,state,postcode,county,region
0,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
1,"39.68,-111.32",nan,Utah,84629,Sanpete County,nan
2,"39.0055,-114.2205",nan,Nevada,nan,White Pine County,nan
3,"37.62544,-120.95492",nan,California,95319,Stanislaus County,nan
4,"39.098,-79.4322",nan,West Virginia,26260,Tucker County,nan
...,...,...,...,...,...,...
4219,"44.35904,-89.83695",nan,Wisconsin,54494,Wood County,nan
4220,"-37.8333,140.7833",Mount Gambier,South Australia,5290,nan,nan
4221,"48.13,-115.62",nan,Montana,nan,Lincoln County,nan
4222,"55.22068,-162.7323",Cold Bay,Alaska,99571,Aleutians East,nan


In [168]:
# Create a lat_lon column which is a concatenated latitude and longitude 
df_moi_date_range_20y = df_moi_date_range_20y\
.assign(lat_lon = lambda df: (df.LATITUDE.astype(str) + "," + df.LONGITUDE.astype(str)))

# Join df_moi_date_range_20y with the lat_lon_df from the reverse geocoding 
df_moi_date_range_20y = df_moi_date_range_20y.merge(lat_lon_df\
                                                    .loc[:, ['lat_lon','city','state', 'postcode', 'county', 'region']], 
                                                    how='left', on='lat_lon')

# Check out the df
df_moi_date_range_20y
                                                    

,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,...,The FTP site ftp://ftp.nodc.noaa.gov/ has been decommissioned,COUNTRY_CODE_2_CHAR,COUNTRY_CODE_3_CHAR,country_name,lat_lon,city,state,postcode,county,region
0,170,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,75.0,...,NaN,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
1,171,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",134.0,0,76.0,...,NaN,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
2,172,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",112.0,0,63.0,...,NaN,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
3,173,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,70.0,...,NaN,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
4,174,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",97.0,0,59.0,...,NaN,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84495,2524916,GME00121978,2017-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",188.0,E,105.0,...,NaN,GM,GME,Germany,"51.4056,6.9683",Essen,Nordrhein-Westfalen,45133,nan,nan
84496,2524917,GME00121978,2018-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",144.0,E,81.0,...,NaN,GM,GME,Germany,"51.4056,6.9683",Essen,Nordrhein-Westfalen,45133,nan,nan
84497,2524918,GME00121978,2019-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",172.0,E,110.0,...,NaN,GM,GME,Germany,"51.4056,6.9683",Essen,Nordrhein-Westfalen,45133,nan,nan
84498,2524919,GME00121978,2020-01-01,51.40560,6.9683,150.0,"ESSEN BREDENEY, GM",166.0,E,92.0,...,NaN,GM,GME,Germany,"51.4056,6.9683",Essen,Nordrhein-Westfalen,45133,nan,nan


## Define US Regions: Northeast, Southeast, Midwest, Southwest, Pacific Northwest

In [169]:
# Create dictionary 
us_regions = {'Northeast': ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Connecticut', 'New York', 
                            'New Jersey', 'Pennsylvania', 'Maryland','Rhode Island', 'Delaware'],
             'Southeast': ['Virginia', 'West Virginia', 'Tennessee', 'North Carolina', 'South Carolina',
                           'Georgia', 'Florida', 'Alabama', 'Mississippi', 'Louisiana', 'Kentucky', 'Arkansas'],
             'Southwest': ['Arizona', 'New Mexico', 'Texas', 'Oklahoma', 'Nevada', 'Colorado', 'Utah', 'California'],
             'Midwest': ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 
                        'Missouri', 'Wisconsin', 'Illinois', 'Michigan', 'Indiana', 'Ohio'],
             'Northwest': ['Washington', 'Oregon', 'Idaho','Montana', 'Wyoming']}

#Convert dictionary to a usable dataframe. 
temp_df = pd.DataFrame(us_regions.values(), index=us_regions.keys()).reset_index()
us_regions_df = temp_df.melt(id_vars=['index'])
us_regions_df = us_regions_df.loc[~us_regions_df['value'].isna() , ['index', 'value']].reset_index(drop = True)\
                .sort_values('index')

us_regions_df.columns = ['cont_us_region', 'state']

# Test to see if any states were missed
set(df_moi_date_range_20y.loc[df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'] == 'US', 'state'].unique()).difference(set(us_regions_df['state']))
    

{'Alaska', 'Hawaii'}

In [171]:
# Join to main df
df_moi_date_range_20y = \
df_moi_date_range_20y\
.merge(us_regions_df, how='left', on = 'state')

df_moi_date_range_20y.head(5)

,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DP01,DP01_ATTRIBUTES,DP10,...,COUNTRY_CODE_2_CHAR,COUNTRY_CODE_3_CHAR,country_name,lat_lon,city,state,postcode,county,region,cont_us_region
0,170,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,75.0,...,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan,Southeast
1,171,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",134.0,0,76.0,...,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan,Southeast
2,172,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",112.0,0,63.0,...,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan,Southeast
3,173,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",117.0,0,70.0,...,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan,Southeast
4,174,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",97.0,0,59.0,...,US,USW,United States,"32.13133,-81.2023",Savannah,Georgia,31418,Chatham County,nan,Southeast


## Export to CSV for Tableau

In [173]:
# Subset Columns to only relevant columns 

column_subset = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 
                 moi, moia, 'COUNTRY_CODE_2_CHAR', 'country_name',
                'city', 'state', 'postcode', 'county', 'cont_us_region']
df_moi_date_range_20y_sub = df_moi_date_range_20y.loc[:, column_subset]
df_moi_date_range_20y_sub.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TAVG,TAVG_ATTRIBUTES,COUNTRY_CODE_2_CHAR,country_name,city,state,postcode,county,cont_us_region
0,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.53,0,US,United States,Savannah,Georgia,31418,Chatham County,Southeast
1,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.10,0,US,United States,Savannah,Georgia,31418,Chatham County,Southeast
2,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.02,0,US,United States,Savannah,Georgia,31418,Chatham County,Southeast
3,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",18.84,0,US,United States,Savannah,Georgia,31418,Chatham County,Southeast
4,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.18,0,US,United States,Savannah,Georgia,31418,Chatham County,Southeast


In [175]:
# Export Subset to CSV to explore in Tableau 
df_moi_date_range_20y_sub.to_csv('df_moi_date_range_20y_sub.csv')

# Linear Regression for US States. 

## Preprocessing (convert years, subset DF)

In [10]:
# Read in CSV
df_moi_date_range_20y_sub = pd.read_csv('../df_moi_date_range_20y_sub.csv')

#See CSV and Columns
df_moi_date_range_20y_sub.head(5)

,Unnamed: 0,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TAVG,TAVG_ATTRIBUTES,COUNTRY_CODE_2_CHAR,country_name,region_x,subregion,city,state,postcode,county,region_y,cont_us_region
0,0,USW00003822,2002-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.53,0,US,United States,GA,Chatham County,Savannah,Georgia,31418,Chatham County,NaN,Southeast
1,1,USW00003822,2003-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.10,0,US,United States,GA,Chatham County,Savannah,Georgia,31418,Chatham County,NaN,Southeast
2,2,USW00003822,2004-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.02,0,US,United States,GA,Chatham County,Savannah,Georgia,31418,Chatham County,NaN,Southeast
3,3,USW00003822,2005-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",18.84,0,US,United States,GA,Chatham County,Savannah,Georgia,31418,Chatham County,NaN,Southeast
4,4,USW00003822,2006-01-01,32.13133,-81.2023,8.7,"SAVANNAH INTERNATIONAL AIRPORT, GA US",19.18,0,US,United States,GA,Chatham County,Savannah,Georgia,31418,Chatham County,NaN,Southeast


In [44]:
# Define start year and create new column for years since 2002
years_since = 2002

# Create a dynamic time difference column
df_moi_date_range_20y_sub['Years since ' + str(years_since)] = pd.to_datetime(df_moi_date_range_20y_sub['DATE']) - datetime(2002,1,1)

# Convert time difference to years (from days)
df_moi_date_range_20y_sub['Years since ' + str(years_since)] = df_moi_date_range_20y_sub['Years since ' + str(years_since)].astype("timedelta64[Y]")


0         0.0
1         0.0
2         1.0
3         3.0
4         4.0
         ... 
84495    15.0
84496    16.0
84497    16.0
84498    17.0
84499    19.0
Name: Years since 2002, Length: 84500, dtype: float64

# (Long run time) Add state, and sub-region information to dataframe using Nominatim (geocoder)

In [ ]:
# Test Case Using a US Lat/Lon

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "Global Summary of the Year Analysis")
location = geolocator.reverse("51.40, 6.9683")
location.raw


# Test Case Using a German Lat/Lon

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "Global Summary of the Year Analysis")
location = geolocator.reverse("32.13, -81.20").raw['address']['city']
location # Test case works!

In [ ]:
# Define a function that returns city, state, postcode, county, and region, if avaiable,
# based on the lat_lon of a station

def reverse_geocode(lat_lon):
    location_details = {'city': np.nan, 'state': np.nan, 'postcode': np.nan, 'county': np.nan, 'region': np.nan}
    geolocator = Nominatim(user_agent = "Global Summary of the Year Analysis")
    location = geolocator.reverse(lat_lon, timeout = 600) #Allows for 10 min, or 600 seconds before timing out
    
    if 'raw' in dir(location):
        location = location.raw['address']
        
        if 'city' in location.keys():
            location_details['city'] = location['city']

        if 'state' in location.keys():
            location_details['state'] = location['state']

        if 'postcode' in location.keys():
            location_details['postcode'] = location['postcode']
        
        if 'county' in location.keys():
            location_details['county'] = location['county']           
          
        if 'region' in location.keys():
            location_details['region'] = location['region']
        
    return location_details

In [ ]:
# Sample subset of rows and test reverse_geocode function

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import swifter

df_moi_date_range_20y_sample = df_moi_date_range_20y.sample(n=250)


df_moi_date_range_20y_sample['CITY'] = df_moi_date_range_20y_sample['lat_lon'].swifter.apply(lambda x: reverse_geocode(x)['city'])
# df_moi_date_range_20y_sample['STATE'] = df_moi_date_range_20y_sample['lat_lon'].apply(lambda x: reverse_geocode(x)['state'])
# df_moi_date_range_20y_sample['POSTCODE'] = df_moi_date_range_20y_sample['lat_lon'].apply(lambda x: reverse_geocode(x)['postcode'])
# df_moi_date_range_20y_sample['COUNTY'] = df_moi_date_range_20y_sample['lat_lon'].apply(lambda x: reverse_geocode(x)['county'])
# df_moi_date_range_20y_sample['REGION'] = df_moi_date_range_20y_sample['lat_lon'].apply(lambda x: reverse_geocode(x)['region'])

#Vectorize geo_location: (this failed)
#df_moi_date_range_20y_sample['CITY'] = reverse_geocode(df_moi_date_range_20y_sample['lat_lon'])['city']




In [ ]:
# Create funtion that adds city, state, postcode, county, region to a dataframe
# Function requires a column 'lat_lon'

def cspcr(dataframe):
    dataframe['CITY'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x)['city'])
    dataframe['STATE'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x)['state'])
    dataframe['POSTCODE'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x)['postcode'])
    dataframe['COUNTY'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x)['county'])
    dataframe['REGION'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x)['region'])
    
    return dataframe

#Alternative approach, append entire dictionary to dataframe, then unpack later to save number of apply calls 
def efficient_cspcr(dataframe):
    dataframe['REVERSE_GEOCODE'] = dataframe['lat_lon'].progress_apply(lambda x: reverse_geocode(x))
    
    return dataframe

In [ ]:
# Use function to get city, state, postcode, county, and region added to temporary lat_lon df

lat_lon_df = pd.DataFrame({'lat_lon': df_moi_date_range_20y['lat_lon'].unique()})

lat_lon_df = efficient_cspcr(lat_lon_df)

lat_lon_df.head(5)

### Write Lat_Lon Geocode Data to CSV

In [ ]:
# Write lat_lon_df to CSV to avoid having to do that again! (took 38 min)

lat_lon_df.to_csv("lat_lon_df.csv")


# Appendix

In [182]:
#### Appendix: Troubleshooting NAs for Postcode/State/City

# Check how many rows have an NA for Postcode + State + City and see if there are any commonalities 

nada = df_moi_date_range_20y_sample\
    .loc[df_moi_date_range_20y_sample\
    .loc[:,['CITY', 'STATE','POSTCODE', 'COUNTY', 'REGION']]\
    .isnull().all(axis = 1) , ['STATION', 'lat_lon']]

#for i in nada.lat_lon:
    #print(geolocator.reverse(i).raw['address'])
    
# Based on this, add county and region to the function 

len(nada)

0

In [22]:
#### Appendix: This approach had too much missing data. Used reverse geocoding instead.


# Read in master location idenfitier database (mlid)
mlid = pd.read_excel('master-location-identifier-database.xlsx', header = 4)


# Join on station code (ghcn) and bring in redion and subregion
df_moi_date_range_20y = df_moi_date_range_20y.merge(mlid.loc[:, ['ghcn','region', 'subregion']] , 
                                                    how='left', left_on='STATION', right_on='ghcn')
                                                    

# What percent of data has region and subregion information?

print('For all countries')
regions = ['region', 'subregion']
for i in regions:
    percent_missing(df_moi_date_range_20y, 'STATION', i)
    
# What percent of US data has region and subregion information?
print('For just US')
regions = ['region', 'subregion']
for i in regions:
    temp_df = df_moi_date_range_20y.loc[(df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'] == 'US'),:]
    percent_missing(temp_df, 'STATION', i)
    
# What percent of NON-US data has region and subregion information?
print('For everywhere but US')
regions = ['region', 'subregion']
for i in regions:
    temp_df = df_moi_date_range_20y.loc[(df_moi_date_range_20y['COUNTRY_CODE_2_CHAR'] != 'US'),:]
    percent_missing(temp_df, 'STATION', i)



,country3,country2,country,region,subregion,place_name,station_name,type,stn_key,status,icao,national_id,wmo,wban,ghcn,special,lat,lon,elev
0,ABW,AW,Aruba,-,-,Old Norwood,John A. Osborne AP,NaN,AWaaNORW,-,NaN,NaN,78850.0,NaN,NaN,NaN,16.791111,-62.193333,166.7
1,ABW,AW,Aruba,SA,-,Upper Hell's Gate,"Juancho E. Yrausquin AP, Saba",NaN,AWaaHLGT,-,NaN,NaN,78871.0,NaN,NaN,NaN,17.646111,-63.220833,42.1
2,AFG,AF,Afghanistan,BAL,-,Kyzylabad|Mazar-I-Sharif|Qizil?b?d,Kyzylabad,NaN,AFaaOAMS,-,OAMS,NaN,40911.0,NaN,NaN,NaN,36.700000,67.200000,392.3
3,AFG,AF,Afghanistan,BAL,-,Pushti-Bag|Mazari Sharif|Dehd?d?,Camp Spann,NaN,AFaaKQSP,-,KQSP,NaN,NaN,NaN,NaN,NaN,36.650400,66.996000,408.0
4,AFG,AF,Afghanistan,BAL,-,Qala-i-Gul Mohd|Mazari Sharif|Qizil?b?d,Camp Marmal,NaN,AFaaKQML,-,KQML,NaN,NaN,NaN,NaN,NaN,36.703000,67.228000,391.0
5,AFG,AF,Afghanistan,BAL,-,Shor-Tipa|Shor Tepe,Shor-Tipa,NaN,AFs40910,-,NaN,NaN,40910.0,NaN,NaN,NaN,37.333000,66.850000,274.0
6,AFG,AF,Afghanistan,BAM,-,Dekhi-Khazara|Bamiyan|Deh-e Haz?rah,Dekhi-Khazara,NaN,AFaaKQC6,-,KQC6,NaN,NaN,NaN,NaN,NaN,34.770000,67.800000,2556.0
7,AFG,AF,Afghanistan,BAM,-,Gorvana|Sar ?sy?b,Gorvana,NaN,AFaaOABN,-,OABN,NaN,40945.0,NaN,NaN,NaN,34.816667,67.816667,2573.0
8,AFG,AF,Afghanistan,BAM,-,Kala-Nau|Pan Jao|Dahan-e M?r,Kala-Nau,NaN,AFaaOAPJ,-,OAPJ,NaN,40946.0,NaN,NaN,NaN,34.398056,67.021944,2813.0
9,AFG,AF,Afghanistan,BAM,-,Tadzhik|Bamiyan PRT|T?jik,Tadzhik,NaN,AFaaKQKP,-,KQKP,NaN,NaN,NaN,NaN,NaN,34.800000,67.800000,2550.0
